In [1]:
import tensorflow as tf
import mtcnn
from mtcnn import MTCNN
import PIL
from PIL import Image
import numpy as np
import pandas as pd
from IPython.display import display
import os
import matplotlib.pyplot as pyplot
from tensorflow.keras.models import load_model
from sklearn.preprocessing import Normalizer, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib 

Using TensorFlow backend.


In [2]:
FaceNet = load_model('../keras-facenet/model/facenet_keras.h5')

In [3]:
SVM_model = joblib.load('../Face Recognition/SVM_model.pkl')

In [4]:
normalizer = Normalizer(norm = 'l2')

In [5]:
name = {0:'Aditya Solanki', 
        1:'Ben Afflek',
        2:'Elton John', 
        3:'Jerry Seinfeld',
        4:'Madonna', 
        5:'Mindy Kaling'}

In [6]:
def predict_face(FaceNet, SVM_model, filename, normalizer):
    image = Image.open(filename)
    #Convert the file to RGB
    image = image.convert('RGB')
    #Convert the File to Numpy array to be machine readable
    pixels = np.asarray(image)
    #Creating a Face Detector using MTCNN
    detector = MTCNN()
    #Extracting Face embeddings from the Photo
    result = detector.detect_faces(pixels)
    #print(result)
    x1, y1, width, height = result[0]['box']
    x1, y1 = abs(x1) , abs(y1)
    x2, y2 = abs(x1) + width , abs(y1) + height 
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize((160,160))
    face_array = np.asarray(image)
    face_pixels = face_array.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels  = (face_pixels - mean)/std
    samples = np.expand_dims(face_pixels, axis = 0)
    output = FaceNet.predict(samples)
    embeddings = np.asarray(output)
    embeddings_normalized = normalizer.transform(embeddings)
    prediction = SVM_model.predict(embeddings_normalized)
    return name[prediction[0]]

In [15]:
filename = ('../data/test/Ben.jpg')
prediction = predict_face(FaceNet,SVM_model, filename, normalizer)

In [16]:
prediction

'Ben Afflek'